In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-09


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (385536, 9)
Loaded df_finviz shape: (1536, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1536


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO   BBJP  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-10  143.10  35.78  13.41  87.53  166.99  163.09  2.3576  160.28  108.95  87.66  20.32  33.06  93.81  321.67  106.53  100.72  51.26  487.22  54.12  193.12  61.56  240.62  239.01  6.28  70.01  5.84  60.61  80.20  84.36  16.89  120.83  81.23  33.57  121.09  92.21  15.12  8.04  73.74  191.00  174.82  238.03  104.38  71.31  125.39  81.92  327.82  41.96  166.40  127.66  36.75  152.41  81.38  116.50  13.32  207.50  8.68  167.14  178.58  258.57  35.02  43.47  420.51  176.00  17.96  185.95  154.43  72.06  339.49  314.01  84.07  33.80  230.07  38.89  56.87  112.64  76.34  225.40  74.95  30.49  18.62  116.83  130.58  382.78  70.15  47.49  125.18  31.55  14.82  968.54  152.58  311.59  2.14  10.64  50.07  112.25  137.79  22.39  3.05  180.27  61.01  62.94  56.73  130.64  62.10  25.56  86.48  88.32  211.88  117.70  115.63  318.26  215.22  63.25  32.55  265.43  47.95  66.74  3025.75  174.63  73.00  35.77  140.61  65.90  39.03  164.68  40.91  45.79  64.60  2.5668  56.08  57.15  27.33  55.39   
2024-04-11  143.93  35.82  13.62  87.28  174.22  161.63  2.3774  165.42  109.22  86.83  20.14  32.85  94.33  322.46  107.16  100.63  51.41  484.28  54.34  195.40  60.29  239.28  241.06  6.27  69.79  5.61  61.01  79.84  83.39  17.02  120.63  78.02  33.80  117.72  92.23  15.33  8.10  72.61  191.41  171.01  236.20  105.09  75.59  128.01  82.04  324.90  42.67  162.35  128.27  36.86  152.09  80.96  116.41  13.21  211.20  8.64  170.50  179.33  262.01  35.05  43.45  417.06  174.34  17.99  189.05  153.37  74.14  340.10  309.05  84.68  33.18  230.40  39.38  57.42  111.46  78.24  224.09  75.77  30.18  18.74  117.91  130.61  379.65  70.51  48.20  131.13  31.60  15.05  986.00  151.04  311.12  2.11  10.89  52.09  111.57  137.53  23.57  3.17  180.12  61.07  63.12  57.11  136.58  62.19  25.55  86.89  88.62  211.37  117.58  115.15  324.09  215.76  60.86  32.53  264.42  47.93  68.05  3006.76  173.36  73.26  35.47  142.08  65.49  38.80  166.12  41.03  45.93  64.29  2.5668  56.12  57.34  28.36  55.86   


In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO   BBJP  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-10  143.10  35.78  13.41  87.53  166.99  163.09  2.3576  160.28  108.95  87.66  20.32  33.06  93.81  321.67  106.53  100.72  51.26  487.22  54.12  193.12  61.56  240.62  239.01  6.28  70.01  5.84  60.61  80.20  84.36  16.89  120.83  81.23  33.57  121.09  92.21  15.12  8.04  73.74  191.00  174.82  238.03  104.38  71.31  125.39  81.92  327.82  41.96  166.40  127.66  36.75  152.41  81.38  116.50  13.32  207.50  8.68  167.14  178.58  258.57  35.02  43.47  420.51  176.00  17.96  185.95  154.43  72.06  339.49  314.01  84.07  33.80  230.07  38.89  56.87  112.64  76.34  225.40  74.95  30.49  18.62  116.83  130.58  382.78  70.15  47.49  125.18  31.55  14.82  968.54  152.58  311.59  2.14  10.64  50.07  112.25  137.79  22.39  3.05  180.27  61.01  62.94  56.73  130.64  62.10  25.56  86.48  88.32  211.88  117.70  115.63  318.26  215.22  63.25  32.55  265.43  47.95  66.74  3025.75  174.63  73.00  35.77  140.61  65.90  39.03  164.68  40.91  45.79  64.60  2.5668  56.08  57.15  27.33  55.39   
2024-04-11  143.93  35.82  13.62  87.28  174.22  161.63  2.3774  165.42  109.22  86.83  20.14  32.85  94.33  322.46  107.16  100.63  51.41  484.28  54.34  195.40  60.29  239.28  241.06  6.27  69.79  5.61  61.01  79.84  83.39  17.02  120.63  78.02  33.80  117.72  92.23  15.33  8.10  72.61  191.41  171.01  236.20  105.09  75.59  128.01  82.04  324.90  42.67  162.35  128.27  36.86  152.09  80.96  116.41  13.21  211.20  8.64  170.50  179.33  262.01  35.05  43.45  417.06  174.34  17.99  189.05  153.37  74.14  340.10  309.05  84.68  33.18  230.40  39.38  57.42  111.46  78.24  224.09  75.77  30.18  18.74  117.91  130.61  379.65  70.51  48.20  131.13  31.60  15.05  986.00  151.04  311.12  2.11  10.89  52.09  111.57  137.53  23.57  3.17  180.12  61.07  63.12  57.11  136.58  62.19  25.55  86.89  88.62  211.37  117.58  115.15  324.09  215.76  60.86  32.53  264.42  47.93  68.05  3006.76  173.36  73.26  35.47  142.08  65.49  38.80  166.12  41.03  45.93  64.29  2.5668  56.12  57.34  28.36  55.86   


df_close_returns shape: (250, 1536)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-11  0.005800  0.001118  0.015660 -0.002856  0.043296 -0.008952  0.008398  0.032069  0.002478 -0.009468 -0.008858 -0.006352  0.005543  0.002456  0.005914 -0.000894  0.002926 -0.006034  0.004065  0.011806 -0.020630 -0.005569  0.008577 -0.001592 -0.003142 -0.039384  0.006600 -0.004489 -0.011498  0.007697 -0.001655 -0.039517  0.006851 -0.027831  0.000217  0.013889  0.007463 -0.015324  0.002147 -0.021794 -0.007688  0.006802  0.060020  0.020895  0.001465 -0.008907  0.016921 -0.024339  0.004778  0.002993 -0.002100 -0.005161 -0.000773 -0.008258  0.017831 -0.004608  0.020103  0.004200  0.013304  0.000857 -0.000460 -0.008204 -0.009432  0.001670  0.016671 -0.006864  0.028865  0.001797 -0.015796  0.007256 -0.018343  0.001434  0.012600  0.009671 -0.010476  0.024889 -0.005812  0.010941 -0.010167  0.006445  0.009244  0.000230 -0.008177  0.005132  0.014951  0.047532  0.001585  0.015520  0.018027 -0.010093 -0.001508 -0.014019  0.023496  0.040344 -0.006058 -0.001887  0.052702  0.039344   
2024-04-12 -0.029459 -0.028476 -0.034508  0.004468  0.008610 -0.022211 -0.020821 -0.036574 -0.016114 -0.014626 -0.013903 -0.025875 -0.019718 -0.030112 -0.015398 -0.011428 -0.017701 -0.021042 -0.009385 -0.032088 -0.025543 -0.002340 -0.016925 -0.015949 -0.004012 -0.010695 -0.003606 -0.009770 -0.012472 -0.054054 -0.008124  0.007947 -0.034911 -0.015291  0.001843 -0.010437 -0.008642 -0.006611 -0.008516  0.003275 -0.015072 -0.015415 -0.037836 -0.059370 -0.012555 -0.030563 -0.032107  0.007207 -0.017931 -0.020347 -0.023604 -0.011981 -0.021905 -0.009084 -0.024053 -0.017361 -0.042346 -0.004405 -0.010076 -0.014836 -0.011738 -0.020788  0.003155 -0.026126 -0.015446  0.004825 -0.085379 -0.017348 -0.009675  0.004133 -0.019892 -0.019965 -0.020823 -0.022118 -0.025570 -0.034126 -0.011870 -0.020853 -0.012922 -0.011740 -0.016623 -0.020289 -0.004293 -0.051340 -0.035062 -0.036605 -0.011076 -0.057143 -0.030578 -0.033104 -0.017582  0.009479 -0.032140 -0.008639 -0.003496 -0.013888 -0.016971 -0.031546   
2024-04-15 -0.003651  0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR       AVB  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       0.001147  0.001963  0.002228  0.001460  0.001736  0.000633  0.000651  0.001527  0.000398  0.000694  0.000238  0.001267  0.000900  0.000869  0.001004  0.000474  0.000800  0.000878  0.000314  0.002062  0.000825  0.000744  0.001097  0.000656  0.000329  0.001136  0.000669  0.000166  0.001033  0.001211  0.000716  0.000709  0.002566  0.001485 -0.000009  0.000869  0.000537  0.000834  0.001303  0.000916  0.000513  0.001309  0.002161  0.002360  0.000968  0.001466  0.002266  0.000601  0.000650  0.001214  0.001250  0.001112  0.001083  0.000584  0.001766  0.000660  0.002438  0.000913  0.000501  0.000790  0.000723  0.001343  0.000166  0.000464  0.001390  0.001080  0.001400  0.000967  0.000440  0.000603  0.002495  0.001088  0.001098  0.001268  0.002071  0.002056  0.000989  0.001322  0.001473  0.001141  0.000776  0.002039  0.000476  0.000676  0.001885  0.002525  0.001204  0.001844  0.001540  0.000372  0.000710  0.002133  0.001467  0.001918  0.000226  0.000799  0.001062  0.002151  0.000892   
AA      0.001963  0.003852  0.004069  0.002609  0.003225  0.001040  0.001239  0.002835  0.000579  0.001201  0.000416  0.002438  0.001652  0.001544  0.001829  0.000829  0.001469  0.001634  0.000555  0.003650  0.001387  0.001306  0.002075  0.001309  0.000528  0.002078  0.001110  0.000207  0.001854  0.002195  0.001278  0.001239  0.004775  0.002796  0.000006  0.001527  0.000969  0.001425  0.002498  0.001695  0.000862  0.002411  0.004072  0.004404  0.001609  0.002780  0.004284  0.001077  0.001156  0.002319  0.002057  0.002094  0.002061  0.001038  0.003190  0.001222  0.004598  0.001652  0.000805  0.001447  0.001335  0.002438  0.000239  0.000820  0.002574  0.002072  0.002474  0.001756  0.000796  0.001120  0.004649  0.001983  0.002013  0.002322  0.003828  0.003985  0.001836  0.002515  0.002669  0.002076  0.001414  0.003732  0.000742  0.001470  0.003496  0.004574  0.002217  0.003549  0.002892  0.000668  0.001360  0.003624  0.002772  0.003517  0.000338  0.001427  0.001956  0.003892  0.001579   


corr_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR       AVB  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.933875  0.871565  0.870653  0.909855  0.715972  0.644926  0.867528  0.620660  0.662689  0.347597  0.878779  0.860383  0.861019  0.926133  0.754220  0.852451  0.770643  0.532308  0.939078  0.763267  0.826601  0.903145  0.738507  0.535756  0.852387  0.694820  0.292340  0.872111  0.858229  0.724762  0.734073  0.863232  0.849552 -0.062316  0.735458  0.763402  0.791155  0.856218  0.811915  0.628953  0.857580  0.838390  0.921095  0.833923  0.874761  0.884691  0.582998  0.832705  0.827799  0.792184  0.853049  0.885623  0.657027  0.926497  0.792052  0.902572  0.917861  0.692776  0.804835  0.842032  0.913694  0.219481  0.540949  0.901177  0.815006  0.780689  0.859939  0.693247  0.847069  0.928590  0.925484  0.886383  0.890898  0.915544  0.674174  0.856828  0.899229  0.856424  0.882123  0.797588  0.890270  0.589926  0.510966  0.915201  0.905653  0.846555  0.864431  0.888035  0.423591  0.672773  0.812012  0.885898  0.911919  0.423046  0.799896  0.788192  0.822943  0.882751   
AA      0.933875  1.000000  0.868636  0.849232  0.922494  0.642726  0.669841  0.879325  0.492707  0.626301  0.332118  0.923122  0.861563  0.835444  0.920443  0.719637  0.853882  0.782948  0.513633  0.907356  0.700651  0.792179  0.932305  0.803922  0.469101  0.851376  0.629564  0.199093  0.854397  0.848983  0.706411  0.700137  0.876585  0.872798  0.021585  0.705532  0.752243  0.737973  0.895798  0.819975  0.576785  0.862187  0.862394  0.938197  0.756691  0.905324  0.913114  0.570558  0.808997  0.862934  0.711572  0.876585  0.920064  0.637815  0.913203  0.800156  0.928977  0.906644  0.607598  0.805115  0.848345  0.905044  0.173088  0.521073  0.910728  0.853518  0.752805  0.851780  0.684523  0.858979  0.944586  0.920589  0.886687  0.890725  0.923446  0.713003  0.868073  0.933899  0.846867  0.876253  0.793533  0.889362  0.502214  0.606405  0.926448  0.895476  0.851312  0.908340  0.909922  0.415570  0.703266  0.753089  0.913674  0.912619  0.345460  0.780090  0.792162  0.812704  0.852616   


In [9]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-09_df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-09_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None